precision 계산

매칭될 title 전처리

In [1]:
import pandas as pd
import re

# CSV 불러오기
t0 = pd.read_csv("t000.csv", encoding='cp949') # 정답셋 title,drug만 있는 거
t1 = pd.read_csv("t4.csv", encoding='utf-8') # 그때그때 추출한 추출셋 

# 1️⃣ title 전처리 함수
def preprocess_title(title):
    if pd.isna(title):
        return ""
    title = title.lower()  # 소문자 통일
    title = re.sub(r"[\-;,/():]", " ", title)  # 특수문자 제거
    title = re.sub(r"\s+", " ", title)  # 공백 통일
    title = title.strip()
    return title

# 2️⃣ t0, t1의 title 컬럼 전처리
t0["title"] = t0["title"].apply(preprocess_title)
t1["title"] = t1["title"].apply(preprocess_title)

# 3️⃣ 전처리 확인 (상위 5개)
print("=== 정답셋 전처리된 title 예시 ===")
print(t0["title"].head())
print("\n=== 예측셋 전처리된 title 예시 ===")
print(t1["title"].head())


=== 정답셋 전처리된 title 예시 ===
0    conditionally reprogrammed colorectal cancer c...
1    combining mek and src inhibitors for treatment...
2    sensitivity of kras mutant colorectal cancers ...
3    targeting ras mutant colorectal cancer with du...
4    mcl 1 inhibition overcomes intrinsic and acqui...
Name: title, dtype: object

=== 예측셋 전처리된 title 예시 ===
0    wee1 confers resistance to krasg12c inhibitors...
1    palbociclib overcomes afatinib resistance in n...
2    regorafenib suppresses epidermal growth factor...
3    her2 exon 20 insertions in non small cell lung...
4    xie bai san increases nsclc cells sensitivity ...
Name: title, dtype: object


자카트르 유사도 title 매칭: 기준 .7

In [2]:
import pandas as pd
import re

# CSV 불러오기
# t0 = pd.read_csv("t0.csv")
# t1 = pd.read_csv("t1.csv")

# 1️⃣ title 전처리
def preprocess_title(title):
    if pd.isna(title):
        return ""
    title = title.lower()  # 소문자 통일
    title = re.sub(r"[\-;,/():]", " ", title)  # 특수문자 제거
    title = re.sub(r"\s+", " ", title)  # 공백 통일
    title = title.strip()
    return title

t0["title"] = t0["title"].apply(preprocess_title)
t1["title"] = t1["title"].apply(preprocess_title)

# 2️⃣ Jaccard 유사도 함수
def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    if not set1 or not set2:
        return 0
    return len(set1 & set2) / len(set1 | set2)

threshold = 0.7  # 유사도 기준 (조정 가능)

# 3️⃣ title 유사도 기반 매칭
matches = []
for i, row0 in t0.iterrows():
    for j, row1 in t1.iterrows():
        sim = jaccard_similarity(row0["title"], row1["title"])
        if sim >= threshold:
            matches.append({
                "title_t0": row0["title"],
                "title_t1": row1["title"],
                "similarity": sim
            })

matched_df = pd.DataFrame(matches)

# 4️⃣ 결과 확인
print(f"매칭된 title 개수: {matched_df.shape[0]}")
print("\n=== Title 유사도 매칭 결과 (상위 10개) ===")
# print(matched_df.head(10))
print(matched_df)


매칭된 title 개수: 33

=== Title 유사도 매칭 결과 (상위 10개) ===
                                             title_t0  \
0   sensitivity of kras mutant colorectal cancers ...   
1   phase ib study of vemurafenib in combination w...   
2   regorafenib suppresses epidermal growth factor...   
3   regorafenib inhibits colorectal tumor growth t...   
4   targeting methionine addiction combined with l...   
5   combining neratinib with cdk4 6 mtor and mek i...   
6   expression of mt4 mmp egfr and rb in triple ne...   
7   doublet and triplet combinations of eribulin f...   
8   carbon nanotube capsules enhance the in vivo e...   
9   dual mtor kinase inhibitor mln0128 sensitizes ...   
10  acquired resistance to endocrine treatments is...   
11  3 dimensional patient derived lung cancer assa...   
12  preclinical antitumor efficacy of bay 1129980 ...   
13  the fact inhibitor cbl0137 synergizes with cis...   
14  patient derived non small cell lung cancer xen...   
15  differential antitumor activity o

드러그 전처리

In [3]:
import pandas as pd
import re

# CSV 불러오기
# t0 = pd.read_csv("t0.csv", encoding='cp949')
# t1 = pd.read_csv("t1.csv", encoding='utf-8')

# 1️⃣ Title 전처리 함수
def preprocess_title(title):
    if pd.isna(title):
        return ""
    title = title.lower()
    title = re.sub(r"[\-;,/():]", " ", title)
    title = re.sub(r"\s+", " ", title)
    return title.strip()

t0["title"] = t0["title"].apply(preprocess_title)
t1["title"] = t1["title"].apply(preprocess_title)

# 2️⃣ Jaccard 유사도 함수
def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    if not set1 or not set2:
        return 0
    return len(set1 & set2) / len(set1 | set2)

threshold = 0.7  # 유사도 기준

# 3️⃣ Title 유사도 기반 매칭
matches = []
for i, row0 in t0.iterrows():
    for j, row1 in t1.iterrows():
        sim = jaccard_similarity(row0["title"], row1["title"])
        if sim >= threshold:
            matches.append({
                "title_t0": row0["title"],
                "title_t1": row1["title"],
                "similarity": sim,
                "drug_t0": row0["drug"],
                "drug_t1": row1["drug"]
            })

matched_df = pd.DataFrame(matches)
print(f"매칭된 title 개수: {matched_df.shape[0]}")

# 4️⃣ Drug 전처리
def preprocess_drug(drug_str):
    if pd.isna(drug_str):
        return []
    return [d.strip().lower() for d in drug_str.split(";") if d.strip()]

matched_df["drug_t0_list"] = matched_df["drug_t0"].apply(preprocess_drug)
matched_df["drug_t1_list"] = matched_df["drug_t1"].apply(preprocess_drug)

# 5️⃣ Precision 계산: pred(t1) 기준
def compute_precision(row):
    set_pred = set(row["drug_t1_list"])
    set_true = set(row["drug_t0_list"])
    if len(set_pred) == 0:
        return 0
    return len(set_pred & set_true) / len(set_pred)

matched_df["row_precision"] = matched_df.apply(compute_precision, axis=1)
precision_macro = matched_df["row_precision"].mean()  # 행별 평균 Precision
precision_micro = matched_df["drug_t1_list"].explode().isin(
    matched_df["drug_t0_list"].explode()
).sum() / matched_df["drug_t1_list"].explode().shape[0]  # 전체 기준 Precision

# 6️⃣ 결과 출력
print("\n=== Drug Precision ===")
print(f"🔹 Macro Precision (행별 평균): {precision_macro:.4f}")
print(f"🔹 Micro Precision (전체 기준): {precision_micro:.4f}")

# 7️⃣ 행별 상세 출력 (옵션)
for idx, row in matched_df.iterrows():
    print("------------------------------------")
    print(f"Title T0: {row['title_t0']}")
    print(f"Title T1: {row['title_t1']}")
    print(f"Drug T0: {row['drug_t0_list']}")
    print(f"Drug T1: {row['drug_t1_list']}")
    print(f"Precision (row): {row['row_precision']}")


매칭된 title 개수: 33

=== Drug Precision ===
🔹 Macro Precision (행별 평균): 0.4899
🔹 Micro Precision (전체 기준): 0.6087
------------------------------------
Title T0: sensitivity of kras mutant colorectal cancers to combination therapy that cotargets mek and cdk4 6
Title T1: sensitivity of kras mutant colorectal cancers to combination therapy that cotargets mek and cdk4 6
Drug T0: ['palbociclib', 'trametinib']
Drug T1: ['trametinib']
Precision (row): 1.0
------------------------------------
Title T0: phase ib study of vemurafenib in combination with irinotecan and cetuximab in patients with metastatic colorectal cancer with brafv600e mutation
Title T1: phase ib study of vemurafenib in combination with irinotecan and cetuximab in patients with metastatic colorectal cancer with brafv600e mutation
Drug T0: ['irinotecan', 'vemurafenib', 'cetuximab']
Drug T1: ['irinotecanirinotecan']
Precision (row): 0.0
------------------------------------
Title T0: regorafenib suppresses epidermal growth factor rece

recall, f1 함께 계산

In [4]:
# ======================================
# 8️⃣ Recall 계산: t0 기준
# ======================================
def compute_recall(row):
    set_pred = set(row["drug_t1_list"])
    set_true = set(row["drug_t0_list"])
    if len(set_true) == 0:
        return 0
    return len(set_pred & set_true) / len(set_true)

matched_df["row_recall"] = matched_df.apply(compute_recall, axis=1)
recall_macro = matched_df["row_recall"].mean()  # 행별 평균 Recall

# Micro Recall (전체 기준)
all_true = matched_df["drug_t0_list"].explode()
all_pred = matched_df["drug_t1_list"].explode()
recall_micro = all_pred.isin(all_true).sum() / all_true.shape[0]

# ======================================
# 9️⃣ F1 Score 계산
# ======================================
def compute_f1(p, r):
    if (p + r) == 0:
        return 0
    return 2 * p * r / (p + r)

matched_df["row_f1"] = matched_df.apply(
    lambda row: compute_f1(row["row_precision"], row["row_recall"]), axis=1
)
f1_macro = matched_df["row_f1"].mean()

# Micro F1 (전체 기준)
if (precision_micro + recall_micro) == 0:
    f1_micro = 0
else:
    f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro)

# ======================================
# 10️⃣ 결과 출력
# ======================================
print("\n=== Drug Precision & Recall & F1 ===")
print(f"🔹 Macro Precision (행별 평균): {precision_macro:.4f}")
print(f"🔹 Micro Precision (전체 기준): {precision_micro:.4f}")
print(f"🔹 Macro Recall (행별 평균): {recall_macro:.4f}")
print(f"🔹 Micro Recall (전체 기준): {recall_micro:.4f}")
print(f"🔹 Macro F1 (행별 평균): {f1_macro:.4f}")
print(f"🔹 Micro F1 (전체 기준): {f1_micro:.4f}")

# 행별 상세 출력 (옵션)
for idx, row in matched_df.iterrows():
    print("------------------------------------")
    print(f"Title T0: {row['title_t0']}")
    print(f"Title T1: {row['title_t1']}")
    print(f"Drug T0: {row['drug_t0_list']}")
    print(f"Drug T1: {row['drug_t1_list']}")
    print(f"Precision (row): {row['row_precision']:.4f}")
    print(f"Recall (row): {row['row_recall']:.4f}")
    print(f"F1 (row): {row['row_f1']:.4f}")



=== Drug Precision & Recall & F1 ===
🔹 Macro Precision (행별 평균): 0.4899
🔹 Micro Precision (전체 기준): 0.6087
🔹 Macro Recall (행별 평균): 0.4051
🔹 Micro Recall (전체 기준): 0.4308
🔹 Macro F1 (행별 평균): 0.3949
🔹 Micro F1 (전체 기준): 0.5045
------------------------------------
Title T0: sensitivity of kras mutant colorectal cancers to combination therapy that cotargets mek and cdk4 6
Title T1: sensitivity of kras mutant colorectal cancers to combination therapy that cotargets mek and cdk4 6
Drug T0: ['palbociclib', 'trametinib']
Drug T1: ['trametinib']
Precision (row): 1.0000
Recall (row): 0.5000
F1 (row): 0.6667
------------------------------------
Title T0: phase ib study of vemurafenib in combination with irinotecan and cetuximab in patients with metastatic colorectal cancer with brafv600e mutation
Title T1: phase ib study of vemurafenib in combination with irinotecan and cetuximab in patients with metastatic colorectal cancer with brafv600e mutation
Drug T0: ['irinotecan', 'vemurafenib', 'cetuximab']

0.5이하 precision 결과 확인하기

In [5]:
# Precision 0.5 이하 필터링
low_precision_df = matched_df[matched_df["row_precision"] <= 0.5]

print(f"🔹 행별 Precision <= 0.5인 개수: {len(low_precision_df)}")
print("\n=== Low Precision Rows ===")
for idx, row in low_precision_df.iterrows():
    print("------------------------------------")
    print(f"Title T0: {row['title_t0']}")
    print(f"Title T1: {row['title_t1']}")
    print(f"Drug T0: {row['drug_t0_list']}")
    print(f"Drug T1: {row['drug_t1_list']}")
    print(f"Precision (row): {row['row_precision']:.4f}")


🔹 행별 Precision <= 0.5인 개수: 17

=== Low Precision Rows ===
------------------------------------
Title T0: phase ib study of vemurafenib in combination with irinotecan and cetuximab in patients with metastatic colorectal cancer with brafv600e mutation
Title T1: phase ib study of vemurafenib in combination with irinotecan and cetuximab in patients with metastatic colorectal cancer with brafv600e mutation
Drug T0: ['irinotecan', 'vemurafenib', 'cetuximab']
Drug T1: ['irinotecanirinotecan']
Precision (row): 0.0000
------------------------------------
Title T0: regorafenib suppresses epidermal growth factor receptor signaling modulated progression of colorectal cancer
Title T1: regorafenib suppresses epidermal growth factor receptor signaling modulated progression of colorectal cancer
Drug T0: ['regorafenib']
Drug T1: ['regorafenib', 'erlotinib', '6bromo(2chlorophenyl)quinazoline2one']
Precision (row): 0.3333
------------------------------------
Title T0: carbon nanotube capsules enhance the